In [81]:
import pandas as pd
import numpy as np
import datetime

In [94]:
xls = pd.ExcelFile('Data23.xlsx')
sales = pd.read_excel(xls, 'SALES')
products = pd.read_excel(xls, 'PRODUCTS')
customers = pd.read_excel(xls, 'CUSTOMERS')

In [95]:
final =  sales.merge(products,on='ProductID').merge(customers,on='UserID')

In [84]:
final.head()

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,Gender,Location,Age
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30
1,8317,500546547,2017-03-30 23:53:07.050,10334,MOBILE,Cash,21.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30
2,45056,500546547,2018-02-03 18:32:01.840,10173,MOBILE,Mobile Payment,66.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30
3,47085,500546547,2018-02-17 18:03:09.930,10173,MOBILE,Mobile Payment,96.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30
4,52212,500546547,2018-03-22 12:04:56.920,10173,MOBILE,Cash,45.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30


In [96]:
final.dtypes


TransactionID                    int64
UserID                           int64
DateTime                datetime64[ns]
ProductID                        int64
Channel                         object
PaymentType                     object
Price                          float64
Discount                        object
Category                        object
UserFirstTransaction    datetime64[ns]
Gender                          object
Location                        object
Age                              int64
dtype: object

In [86]:
# I found that playing with dtypes messes up the code. I just kept them as datetime instead of trying to transform to date.

# final['DateTime'] = pd.to_datetime(final['DateTime']).dt.date
# final['UserFirstTransaction'] = pd.to_datetime(final['UserFirstTransaction']).dt.date


In [97]:
print(final['DateTime'].min())
print(final['DateTime'].max())
print(final['DateTime'].max() - final['DateTime'].min())

2017-01-01 01:40:39.180000
2018-06-30 23:56:14.570000
545 days 22:15:35.390000


In [88]:
final.dtypes

TransactionID             int64
UserID                    int64
DateTime                 object
ProductID                 int64
Channel                  object
PaymentType              object
Price                   float64
Discount                 object
Category                 object
UserFirstTransaction     object
Gender                   object
Location                 object
Age                       int64
dtype: object

In [89]:
# I don't think this step is necessary. We can do it as the final step instead
# first9 = first9.set_index('UserID')


In [98]:
first9 = final.loc[final['DateTime'] < pd.Timestamp(2017,10,1), ]
last9 = final.loc[final['DateTime'] >= pd.Timestamp(2017,10,1), ]

# print(first9["UserID"].mode)
# print(last9.shape)

last_transaction = first9.groupby("UserID")["DateTime"].agg(max)
# last_transaction.index[1]
# We either do the following when UserID is set as index or we can put a boolean in the row part of loc to yield the same results
for i, user in enumerate(last_transaction):
  first9.loc[first9["UserID"] == last_transaction.index[i], "UserLastTransaction"] = user

first9.head()

<ipython-input-98-6f79be4834e5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first9.loc[first9["UserID"] == last_transaction.index[i], "UserLastTransaction"] = user


,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,Gender,Location,Age,UserLastTransaction
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.00,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30,2017-09-30 19:43:21.230
1,8317,500546547,2017-03-30 23:53:07.050,10334,MOBILE,Cash,21.00,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30,2017-09-30 19:43:21.230
5,2736,500546547,2017-01-30 16:34:18.120,10506,MOBILE,Cash,41.70,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30,2017-09-30 19:43:21.230
6,1012,500546547,2017-01-11 20:08:00.490,10398,MOBILE,Mobile Payment,59.70,Yes,Smart Phones,2015-03-18,FEMALE,ANKARA,30,2017-09-30 19:43:21.230
7,19747,500546547,2017-07-12 17:24:59.400,10398,MOBILE,Mobile Payment,68.85,Yes,Smart Phones,2015-03-18,FEMALE,ANKARA,30,2017-09-30 19:43:21.230


In [99]:
# We want to create the Churn column according to the last transaction date

first9["Churn"] = np.where((first9["UserLastTransaction"] < pd.Timestamp(2017,7,1)), 1, 0)

first9['Churn'].value_counts()



<ipython-input-99-4a26ee92831f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first9["Churn"] = np.where((first9["UserLastTransaction"] < pd.Timestamp(2017,7,1)), 1, 0)


0    26979
1     2567
Name: Churn, dtype: int64

In [92]:
# I have no idea why this doesn't work.
first9['churn'] = np.where(((first9['DateTime'].min() < pd.Timestamp(2017,7,1)) & (first9['DateTime'].max() < pd.Timestamp(2017,7,1))),1,0)

first9['churn'].value_counts()

<ipython-input-92-2f8fe44ec2e5>:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  first9['churn'] = np.where(((first9['DateTime'].min() < pd.Timestamp(2017,7,1)) & (first9['DateTime'].max() < pd.Timestamp(2017,7,1))),1,0)
<ipython-input-92-2f8fe44ec2e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first9['churn'] = np.where(((first9['DateTime'].min() < pd.Timestamp(2017,7,1)) & (first9['DateTime'].max() < pd.Timestamp(2017,7,1))),1,0)


0    29546
Name: churn, dtype: int64

In [100]:
first9

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,Gender,Location,Age,UserLastTransaction,Churn
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.00,No,Female Shoes,2015-03-18 00:00:00.000,FEMALE,ANKARA,30,2017-09-30 19:43:21.230,0
1,8317,500546547,2017-03-30 23:53:07.050,10334,MOBILE,Cash,21.00,No,Female Shoes,2015-03-18 00:00:00.000,FEMALE,ANKARA,30,2017-09-30 19:43:21.230,0
5,2736,500546547,2017-01-30 16:34:18.120,10506,MOBILE,Cash,41.70,No,Sport Shoes,2015-03-18 00:00:00.000,FEMALE,ANKARA,30,2017-09-30 19:43:21.230,0
6,1012,500546547,2017-01-11 20:08:00.490,10398,MOBILE,Mobile Payment,59.70,Yes,Smart Phones,2015-03-18 00:00:00.000,FEMALE,ANKARA,30,2017-09-30 19:43:21.230,0
7,19747,500546547,2017-07-12 17:24:59.400,10398,MOBILE,Mobile Payment,68.85,Yes,Smart Phones,2015-03-18 00:00:00.000,FEMALE,ANKARA,30,2017-09-30 19:43:21.230,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68828,24753,500306098,2017-08-22 14:49:40.340,10340,WEB,Online Credit Card,33.00,No,Computers & Laptops,2017-08-16 18:30:10.850,FEMALE,ISTANBUL,22,2017-08-22 14:49:40.340,0
68829,21488,500470328,2017-07-27 14:53:48.190,10457,MOBILE,Cash,38.25,Yes,Female Fashion,2017-07-27 14:53:48.190,MALE,BURSA,28,2017-07-27 15:50:52.830,0
68830,21497,500470328,2017-07-27 15:50:52.830,10457,MOBILE,Cash,75.00,Yes,Female Fashion,2017-07-27 14:53:48.190,MALE,BURSA,28,2017-07-27 15:50:52.830,0
68845,26609,500320255,2017-09-06 15:57:40.890,10303,MOBILE,Mobile Payment,50.85,No,Female Fashion,2017-09-06 15:57:40.890,MALE,KAYSERI,49,2017-09-06 15:57:40.890,0
